In [24]:
import torch
import os
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from echotorch.nn import ESN, LiESN
from echotorch.utils.matrix_generation import *
import numpy as np
from scipy.special import softmax
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
import torchvision
import pathlib
import librosa
import librosa.display
import glob
from mido import MidiFile, MidiTrack, Message
from statistics import mean, stdev

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [25]:
data_path = '../Hyperdimensional/Datasets/Unprocessed/Chorales/MIDI'
img_path = '../HyperDimensional/img_data'

In [67]:
def process_notes():
	train_chorales = {}
	test_chorales = {}
	UCI_files = glob.glob('../HyperDimensional/Datasets/Unprocessed/Chorales/MIDI/UCI/*.mid')
	rand_train_files = glob.glob('../HyperDimensional/Datasets/Unprocessed/Chorales/MIDI/RandomChorales_train/*.mid')
	rand_test_files = glob.glob('../HyperDimensional/Datasets/Unprocessed/Chorales/MIDI/RandomChorales_test/*.mid')
	boring_files = glob.glob('../HyperDimensional/Datasets/Unprocessed/Chorales/MIDI/BoringChorales/*.mid')
	composed_files = glob.glob('../HyperDimensional/Datasets/Unprocessed/Chorales/MIDI/ComposedChorales/*.mid')
	min_len = 100

	# Chorale files
	for chorale_num, chorale in enumerate(UCI_files):
		chorale = MidiFile(chorale)
		notes = []

		for num, message in enumerate(chorale.tracks[0]):
			if message.type == 'note_on' and message.velocity != 0:
				notes.append(message.note)


		filename = chorale.filename.split('\\')[1].split('.mid')[0]
		train_chorales[filename] = notes
		test_chorales[filename] = notes

		if len(notes) < min_len:
			min_len = len(notes)

	# Random train files
	for chorale_num, chorale in enumerate(rand_train_files):
		chorale = MidiFile(chorale)
		notes = []

		for num, message in enumerate(chorale.tracks[0]):
			if message.type == 'note_on' and message.velocity != 0:
				notes.append(message.note)


		filename = chorale.filename.split('\\')[1].split('.mid')[0]
		train_chorales[filename] = notes

		if len(notes) < min_len:
			min_len = len(notes)

	# Random test files
	for chorale_num, chorale in enumerate(rand_test_files):
		chorale = MidiFile(chorale)
		notes = []

		for num, message in enumerate(chorale.tracks[0]):
			if message.type == 'note_on' and message.velocity != 0:
				notes.append(message.note)

		filename = chorale.filename.split('\\')[1].split('.mid')[0]
		test_chorales[filename] = notes

		if len(notes) < min_len:
			min_len = len(notes)

	# Boring test files
	for chorale_num, chorale in enumerate(boring_files):
		chorale = MidiFile(chorale)
		notes = []

		for num, message in enumerate(chorale.tracks[0]):
			if message.type == 'note_on' and message.velocity != 0:
				notes.append(message.note)


		filename = chorale.filename.split('\\')[1].split('.mid')[0]
		test_chorales[filename] = notes

		if len(notes) < min_len:
			min_len = len(notes)

	# Composed test files
	for chorale_num, chorale in enumerate(composed_files):
		chorale = MidiFile(chorale)
		notes = []

		for num, message in enumerate(chorale.tracks[0]):
			if message.type == 'note_on' and message.velocity != 0:
				notes.append(message.note)


		filename = chorale.filename.split('\\')[1].split('.mid')[0]
		test_chorales[filename] = notes

		if len(notes) < min_len:
			min_len = len(notes)

	for key, value in train_chorales.items():
		train_chorales[key] = torch.Tensor(value[:min_len])
	for key, value in test_chorales.items():
		test_chorales[key] = torch.Tensor(value[:min_len])

	train_chorales = torch.stack([chorale for chorale in train_chorales.values()])

	return train_chorales, test_chorales, min_len


train_chorales, test_chorales, min_len = process_notes()

KeyboardInterrupt: 

In [60]:
# stacked = torch.cat((UCI_chorales, rand_chorales))
labels = torch.cat((torch.ones((1, 100, 1)), torch.zeros((1, 100, 1))), dim=1)

In [21]:
batch_size = 32
im_size = 512
CHORALE_IDX = 0
NCHORALE_IDX = 1

In [25]:
def sigmoid(x):
	return 1 / (1 + np.exp(-x))

def test_model(model, loader):
	results = {}

	for num, (img, label) in enumerate(loader):
		result = model(img.view(1, -1, min_len).to(torch.float).to(device))
		prediction = result.cpu().numpy().flatten()
		results[label[0]] = sigmoid(prediction)

	return results

In [26]:
def train_model(res_size):
	torch.cuda.empty_cache()
	esn = LiESN(
		input_dim=min_len,
		hidden_dim=res_size,
		output_dim=1,
		w_generator=UniformMatrixGenerator(),
		wbias_generator=UniformMatrixGenerator(),
		win_generator=UniformMatrixGenerator(),
		learning_algo='pinv',
		nonlin_func=torch.sigmoid,
		leaky_rate=0.9,
		ridge_param=0.25
	).to(device)

	# for input, label in train_loader:
	input = stacked.view(1, -1, min_len).to(torch.float).to(device)
	label = train_labels.view(1, -1, 1).to(torch.float).to(device)
	esn(input, label)
	return esn

In [ ]:
def print_scores(results):
	mean_scores = {'boring': 0, 'random': 0, 'comp': 0, 'chorale': 0}
	totals = {'boring': 0, 'random': 0, 'comp': 0, 'chorale': 0}
	for (file, score) in results.items():
		if file.startswith('rand'):
			totals['random'] += 1
			mean_scores['random'] += score.item()
		if file.startswith('boring'):
			totals['boring'] += 1
			mean_scores['boring'] += score.item()
		if file.startswith('comp'):
			totals['comp'] += 1
			mean_scores['comp'] += score.item()
		else:
			totals['chorale'] += 1
			mean_scores['chorale'] += score.item()
	
	for key in ['boring', 'random', 'comp', 'chorale']:
		mean_scores[key] /= totals[key]
	
	if totals['boring'] > 0:
		print(f"Boring: {round(mean_scores['boring'],5)}")
	if totals['random'] > 0:
		print(f"Random: {round(mean_scores['random'],5)}")
	if totals['comp'] > 0:
		print(f"Composed: {round(mean_scores['comp'],5)}")
	if totals['chorale'] > 0:
		print(f"Chorale: {round(mean_scores['chorale'],5)}")

	return mean_scores

def print_results(results):
	predictions = {}
	for (file, result) in results.items():
		predictions[file] = softmax(result).flatten()
		predictions[file] = 'Chorale' if np.argmax(predictions[file], 0) == CHORALE_IDX else 'Not chorale'

	accuracy = {'boring': 0, 'random': 0, 'comp': 0, 'chorale': 0}
	totals = {'boring': 0, 'random': 0, 'comp': 0, 'chorale': 0}
	for (file, pred) in predictions.items():
		if file.startswith('rand'):
			totals['random'] += 1
			accuracy['random'] += pred == 'Not chorale'
		if file.startswith('boring'):
			totals['boring'] += 1
			accuracy['boring'] += pred == 'Not chorale'
		if file.startswith('comp'):
			totals['comp'] += 1
			accuracy['comp'] += pred == 'Chorale'
		else:
			totals['chorale'] += 1
			accuracy['chorale'] += pred == 'Chorale'

	if totals['boring'] > 0:
		print(f"Classified {accuracy['boring']}/{totals['boring']} ({round(100*accuracy['boring']/totals['boring'],2)}%) boring compositions as not chorales")
	if totals['random'] > 0:
		print(f"Classified {accuracy['random']}/{totals['random']} ({round(100*accuracy['random']/totals['random'],2)}%) random compositions as not chorales")
	if totals['comp'] > 0:
		print(f"Classified {accuracy['comp']}/{totals['comp']} ({round(100*accuracy['comp']/totals['comp'],2)}%) composition as chorales")
	if totals['chorale'] > 0:
		print(f"Classified {accuracy['chorale']}/{totals['chorale']} ({round(100*accuracy['chorale']/totals['chorale'],2)}%) dataset chorales as chorales")

	return accuracy, totals